In [1]:


import mobster as mb





Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
data = mb.load_example_data("./")

#data = data["1:1"]

In [3]:
#data = { "1:1" : data}

In [6]:
fitted = mb.fit_mobster(data, 1,1,True, max_it = 200, lr = 0.1,  e = 0.00001, lrd_gamma = 0.1,   
        number_of_trials_k= 800, prior_lims_k=[100,10000])

Running MOBSTER on 3 karyotypes with 1 subclones.
Fitting a model with tail

Computing cluster assignements.
Computing information criteria.
Done!


ELBO: -4805.549160671  :  12%|█▏        | 23/200 [00:00<00:05, 31.32it/s]


In [7]:
mb.plot_results(data, fitted, output= "cane.png", drivers=None)

In [9]:
fitted["model_parameters"]["1:1"]



{'cluster_probs': array([[0.0000000e+00, 9.9999845e-01, 1.0000000e+00, ..., 0.0000000e+00,
         1.0000000e+00, 0.0000000e+00],
        [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.7246192e-14,
         0.0000000e+00, 1.0000000e+00],
        [0.0000000e+00, 1.5628425e-06, 1.3151456e-25, ..., 6.2069166e-23,
         4.0646544e-22, 0.0000000e+00],
        [2.2063994e-24, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
         0.0000000e+00, 7.8615097e-20]], dtype=float32),
 'cluster_assignments': array(['C1', 'Tail', 'Tail', ..., 'S2', 'Tail', 'C1'], dtype=object),
 'cluster_types': array([], dtype=object),
 'mixture_probs': array([0.499945  , 0.16670947, 0.16667278, 0.16667278], dtype=float32),
 'beta_concentration1': array([239.9873 , 123.58945, 241.24413], dtype=float32),
 'beta_concentration2': array([259.95767, 676.32245, 558.6678 ], dtype=float32),
 'tail_shape': array(1.0000489, dtype=float32),
 'tail_scale': 0.044045046,
 'tail_noise': 0.020002197507050335}

0.01

1